In [1]:
import os
import re
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [81]:
def hierarchical_markdown_split(md_text: str, path_prefix: str = "") -> list[Document]:
    """마크다운 문서를 계층적으로 분할합니다."""
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#", "title"),
        ("##", "section"),
        ("###", "subsection"),
        ("####", "subsubsection"),
        ("#####", "subsubsubsection")
    ])
    docs = splitter.split_text(md_text)

    result_docs = []
    current_title = None
    chunk_idx = 0
    for doc in docs:
        metadata = doc.metadata
        if "title" in metadata:
            current_title = metadata["title"]

        if current_title:
            chunk_idx += 1
            full_title = "" + current_title
            if "section" in metadata:
                full_title += f" / {metadata['section']}"
            if "subsection" in metadata:
                full_title += f" / {metadata['subsection']}"
            if "subsubsection" in metadata:
                full_title += f" / {metadata['subsubsection']}"
            if "subsubsubsection" in metadata:
                full_title += f" / {metadata['subsubsubsection']}"

            content = f"[section_path]: {full_title}\n\n{doc.page_content}"
            doc = Document(page_content=content, metadata={
                **doc.metadata,
                "type": "documentation",
                "source": "dev_center_guide_allmd.md",
                "chunk_idx": chunk_idx
            })

        result_docs.append(doc)

    return result_docs

def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 마크다운 파일 로드 및 분할
# allmd 적용시 검색 품질의 급격한 저하 확(495 chunks)
str_md_file = load_markdown_file("data/dev_center_guide_touched.md") 
docs_markdown = hierarchical_markdown_split(str_md_file)

print(f"마크다운 문서 분할 완료: {len(docs_markdown)}개 청크")

마크다운 문서 분할 완료: 119개 청크


In [82]:
# docs_markdown에서 'launchPurchaseFlow()' 문자열을 포함하는 Documnet 를 출력

cnt = 0
for doc in docs_markdown:
    if 'PNS' in doc.page_content:
        # print(f"문서 제목: {doc.metadata.get('title', '제목 없음')}")
        print(f"--- doc_index: {cnt} ---")
        print(f"{doc.page_content}[EOD]")  # 처음 200자만 출력
        # print("-" * 40)  # 구분선 
        cnt += 1
        
print(f"'PNS' 문자열을 포함하는 문서 개수: {cnt}")
        

--- doc_index: 0 ---
[section_path]: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 07. PNS (Payment Notification Service) 이용하기 / 개요

원스토어는 개발자를 위해 두 가지 Payment Notification Service를 제공합니다.
* PNS는 Payment Notification Service의 약자입니다.
* PNS는 모바일의 네트워크 연결 불안정성을 보완하기 위해 개발사가 지정한 서버로 개별 사용자의 결제 상태(결제 완료, 결제 취소)를 메시지로 전송하는 기능입니다.
* 정확히는 개발사가 지정한 서버에서 원스토어가 정의한 규칙에 맞추어 API를 구현하면 해당 API를 원스토어의 결제 담당 서버에서 호출하는 형태입니다.
* Server to Server라고 할지라도 네트워크 문제로 메세지 전송 실패가 발생하기 때문에 200 OK로 응답을 인지하지 못할 경우 반복하여 메시지가 전송될 수 있습니다.
아래의 결제 트랜젝션 변화에 대하여 Notification 메시지가 전송됩니다.  
- 인앱상품 결제 또는 결제취소가 발생하면 원스토어가 개발사 서버로 알림을 전송하는 PNS(Payment Notification Service)  
- 구독 상태가 변경되면 개발사 서버로 알림을 전송하는 SNS (Subscription Notifacation Service)  
> Notification은 발송/수신 서버의 상태에 따라 지연 또는 유실될 수 있으므로, notification 수신을 기준으로 상품(서비스)을 제공하는 것은 권장하지 않습니다.
정상적인 결제 건인지 Server to Server로 확인하기를 원하신다면 PNS notification을 이용하는 대신, 관련 서버 API로 조회하는 것을 권장합니다.
원스토어는 검증 및 모니터링 목적으로 결제 테스트를 진행할 수 있으며, 해당 테스트 건들도 결제/결제취소 시 동일하게 notifica

In [105]:
# fixed_model_name = "deepseek-coder:6.7b"
fixed_model_name = "exaone3.5:latest"
# fixed_model_name = "mistral:latest"
# fixed_model_name = "llama3:8b"

def embed_and_save(docs: List[Document], output_path: str):
    """문서를 임베딩하고 FAISS 데이터베이스로 저장합니다."""
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=fixed_model_name)
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")

# 모든 문서 통합
total_docs = docs_markdown #+ docs_kotlin
print(f"총 {len(total_docs)}개의 문서를 생성하였습니다.")
print(f"- 마크다운 문서: {len(docs_markdown)}개")
#print(f"- Kotlin 코드: {len(docs_kotlin)}개")

# 임베딩 생성 및 저장
output_dir = "models/faiss_vs_rag_iap_v2_11_" + fixed_model_name[:3]
os.makedirs(output_dir, exist_ok=True)
embed_and_save(total_docs, output_dir)

총 119개의 문서를 생성하였습니다.
- 마크다운 문서: 119개
✅ 임베딩 저장 완료: models/faiss_vs_rag_iap_v2_11_exa


In [106]:
###### Retriever Check from FAISS Vector DB ######

from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever


# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model=fixed_model_name)

# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path=output_dir,
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 30, "fetch_k": 70, "lambda_mult": 0.7}
    # search_type="similarity",
    # search_kwargs={"k": 50}
)

bm25 = BM25Retriever.from_documents(
    docs_markdown,
    bm25_params={"k1": 1.5, "b": 0.75}
)

bm25.k = 20

ensembled_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],
    weights=[0.5, 0.5]
)


res = ensembled_retriever.invoke(
    "원스토어 인앱결제의 PNS의 개념을 설명해주세요"
)

# res = retriever.invoke(
#     "원스토어 인앱결제의 PNS의 개념을 설명해주세요"
# )

print(f"검색된 문서 수: {len(res)}")

idx = 0
for doc in res: 
    print(f"--- doc_index: {idx} ---")
    print(doc.page_content)  # Print first 100 characters of each document
    # print(doc.metadata)
    # print('-' * 40)
    idx += 1

검색된 문서 수: 41
--- doc_index: 0 ---
[section_path]: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 01. 원스토어 인앱결제 개요 / 원스토어 인앱결제란?

원스토어 인앱결제(In-App Purchase, IAP)는 안드로이드 앱 내에 구현된 상품을 원스토어의 인증 및 결제 시스템을 이용하여 사용자에게 판매, 청구하여 개발자에게 정산하는 서비스입니다.  
인앱 상품 결제를 위해 원스토어 서비스(ONE store service, OSS) 앱과 연동이 필요하며, 원스토어 서비스 앱은 원스토어 결제 서버와 연동하여 인앱상품의 결제를 수행합니다.  
원스토어 인앱결제를 적용하기 위해서는 '원스토어 IAP SDK(In-App Purchase Software Development Kit)'를 적용하면 됩니다.
--- doc_index: 1 ---
[section_path]: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 01. 원스토어 인앱결제 개요 / 결제 프로세스

원스토어 인앱결제는 크게 다음의 네 가지 프로세스로 구성되어 있습니다.
--- doc_index: 2 ---
[section_path]: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 02. 사전준비 / 원스토어 앱 설치하기

개발자나 사용자가 원스토어 인앱결제를 이용하기 위해서는 원스토어 앱이 필요합니다.  
- [원스토어 앱 다운로드 안내](https://onestore-dev.gitbook.io/dev/policy/download)
--- doc_index: 3 ---
[section_path]: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 09. 원스토어 인앱결제 릴리즈 노트 / 원스토어 인앱결제 라이브러리 API 

In [115]:
# 1. MultiQueryRetriever - 다양한 쿼리 변형으로 검색
def create_multi_query_retriever(loaded_db, llm):
    """
    하나의 질문을 여러 변형으로 만들어 검색 품질을 향상시킵니다.
    """
    from langchain.retrievers import MultiQueryRetriever
    from langchain_core.prompts import PromptTemplate
    
    query_prompt = PromptTemplate(
        input_variables=["question"],
        template="""주어진 질문을 기반으로 5개의 다른 방식으로 질문을 변형해주세요.
        각 변형은 원래 질문의 의미를 유지하면서 다른 표현을 사용해야 합니다.
        
        원래 질문: {question}
        
        변형된 질문들:"""
    )
    
    return MultiQueryRetriever.from_llm(
        retriever=loaded_db.as_retriever(search_kwargs={"k": 15}),
        llm=llm,
        prompt=query_prompt,
        parser_key="text"
    )
    

# # 2. ContextualCompressionRetriever - 컨텍스트 압축으로 관련성 향상
# def create_contextual_compression_retriever(loaded_db, llm):
#     """
#     검색된 문서에서 질문과 관련된 부분만 추출하여 관련성을 높입니다.
#     """
#     from langchain.retrievers import ContextualCompressionRetriever
#     from langchain.retrievers.document_compressors import LLMChainExtractor
#     from langchain_core.prompts import PromptTemplate
    
#     compressor_prompt = """다음 문서에서 질문과 관련된 정보만 추출해주세요.
#     질문: {question}
#     문서: {context}
#     관련 정보:"""
    
#     compressor = LLMChainExtractor.from_llm(
#         llm=llm,
#         prompt=PromptTemplate.from_template(compressor_prompt)
#     )
    
#     return ContextualCompressionRetriever(
#         base_retriever=loaded_db.as_retriever(search_kwargs={"k": 20}),
#         base_compressor=compressor
#     )
    
# def create_enhanced_ensemble_retriever(loaded_db, docs_markdown, llm):
#     """
#     여러 검색 방식을 조합한 향상된 앙상블 검색기
#     """
#     from langchain.retrievers import EnsembleRetriever, BM25Retriever
    
#     # 기존 검색기들
#     bm25 = BM25Retriever.from_documents(
#         docs_markdown,
#         bm25_params={"k1": 1.5, "b": 0.75}
#     )
#     bm25.k = 10
    
#     vector_retriever = loaded_db.as_retriever(
#         search_type="mmr",
#         search_kwargs={"k": 10, "fetch_k": 70, "lambda_mult": 0.7}
#     )
    
#     # 새로운 검색기들
#     multi_query_retriever = create_multi_query_retriever(loaded_db, llm)
#     contextual_retriever = create_contextual_compression_retriever(loaded_db, llm)
#     # time_weighted_retriever = create_time_weighted_retriever(loaded_db)
    
#     # 앙상블 구성 (가중치 조정)
#     enhanced_ensemble = EnsembleRetriever(
#         retrievers=[
#             bm25,                    # 키워드 기반 검색
#             vector_retriever,        # 벡터 유사도 검색
#             multi_query_retriever,   # 다중 쿼리 검색
#             # contextual_retriever,    # 컨텍스트 압축 검색
#             # time_weighted_retriever  # 시간 가중치 검색
#         ],
#         weights=[0.2, 0.3, 0.5]  # 가중치 조정
#     )
    
#     return enhanced_ensemble

llm = ChatOllama(model=fixed_model_name, temperature=0.3)

# multi_query = create_multi_query_retriever(loaded_db, llm)
# contextual_compression = create_contextual_compression_retriever(loaded_db, llm)
# powerful_retriever = create_enhanced_ensemble_retriever(loaded_db, docs_markdown, llm)
powerful_retriever = create_multi_query_retriever(loaded_db, llm)

res = powerful_retriever.invoke(
    "PNS(Payment Notification Service)의 메세지 규격은 어떻게 되나요?"
)

print(f"검색된 문서 수: {len(res)}")

idx = 0
for doc in res: 
    print(f"--- doc_index: {idx} ---")
    print(doc.page_content)  # Print first 100 characters of each document
    idx += 1
    # print(doc.metadata)
    # print('-' * 40)

검색된 문서 수: 32
--- doc_index: 0 ---
[section_path]: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
- API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 여기를 클릭해주세요.
- 현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요
- [대한민국 외 국가 및 지역 배포를 위한 가이드](https://onestore-dev.gitbook.io/dev/tools/glb)  
If you are comfortable with English, please change the language to English from the upper left side in this page.
--- doc_index: 1 ---
[section_path]: 원스토어 인앱 결제 연동 가이드 / 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드 / 04. 원스토어 인앱결제 SDK를 사용해 구현하기 / 인앱 라이브러리 적용하기 / PurchaseClient 초기화

PurchaseClient는 원스토어 결제 라이브러리와 앱 간의 통신을 위한 기본 인터페이스입니다.  
단일 이벤트에 관한 여러 개의 PurchasesUpdatedListener 콜백이 발생하는 상황을 피할 수 있도록 PurchaseClient 연결을 열어 두는 것을 권장합니다.  
PurchaseClient를 생성하려면 newBuilder()를 사용합니다. 구매 관련 업데이트를 수신하려면 setListener()를 호출하여 PurchasesUpdatedListener에 대한 참조를 전달해야 합니다. 이 리스너는 앱의 모든 구매 관련 업데이트를 수신합니다.
se

In [116]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# RAG 체인 구성 요소
# embedding_model = OllamaEmbeddings(model=fixed_model_name)
llm = ChatOllama(model=fixed_model_name, temperature=0.3)

# 검색기 설정
# retriever = loaded_db.as_retriever(
#     search_type="mmr",
#     search_kwargs={"k": 30, "fetch_k": 100, "lambda_mult": 0.6}
# )
retriever = create_multi_query_retriever(loaded_db, llm)
# retriever = create_enhanced_ensemble_retriever(loaded_db, docs_markdown, llm)

# 프롬프트 템플릿
prompt_template = """
당신은 원스토어 인앱결제 전문가입니다. 학습한 문서를 바탕으로 질문에 답변해주세요.

답변 시 다음 사항을 고려해주세요:
0. 주어진 문서의 내용을 토대로 답변하고 관련 내용을 발견할 수 없을 경우 '답변을 생성할 수 없습니다'로 답변주세요.
1. 한국어로 명확하고 이해하기 쉽게 답변하세요
2. 코드 예시가 있다면 포함해주세요
3. 개발자 관점에서 실용적인 정보를 제공해주세요
4. 학습한 문서에 정의된 코드값이나 용어가 아닌경우 답변에 포함되어서는 안됩니다.

컨텍스트:
{context}

질문: {question}

답변:"""

prompt = PromptTemplate.from_template(prompt_template)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인이 생성되었습니다!")

✅ RAG 체인이 생성되었습니다!


In [117]:
# 테스트 질문들
test_questions = [
    # "원스토어 인앱결제를 위한 전반적인 절차를 알려주세요",
    # "원스토어 인앱결제 초기화는 어떻게 하나요?",
    # "정기결제 구현 방법을 알려주세요",
    # "PNS(Payment Notification Service) 설정 방법은?",
    "결제 검증은 어떻게 하나요?",
    # "관리형 상품과 구독형 상품의 차이점은?",
    "원스토어의 PNS(Payment Notification Service)란 무엇이고 어떻게 구현하나요? 메시지 수신 서버의 관점에서 Java SpringFramework로 구현한 코드를 예제를 알려주세요",
    "PNS(Payment Notification Service)의 메세지 규격은 어떻게 되나요?",
    # "원스토어 PNS의 JSON 메세지는 어떤 것들이 있나요?"
]

print("🤖 RAG 체인 테스트 시작\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 질문 {i}: {question}")
    print("-" * 60)
    
    try:
        answer = rag_chain.invoke(question)
        print(answer)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("=" * 80)

🤖 RAG 체인 테스트 시작


📝 질문 1: 결제 검증은 어떻게 하나요?
------------------------------------------------------------
결제 검증은 주로 서버 측에서 이루어집니다. 사용자가 구매한 상품에 대한 결제 정보를 서버로 전송하면, 서버는 해당 결제 정보를 검증합니다. 이 검증 과정에서 서버는 결제 정보의 유효성을 확인하고, 결제가 성공적으로 이루어졌는지 확인합니다. 만약 결제가 성공적으로 이루어졌다면, 서버는 해당 결제 정보에 대한 응답을 클라이언트에게 전달하고, 클라이언트 앱은 이를 사용자에게 보여줍니다. 이 과정을 통해 사용자는 결제가 정상적으로 이루어졌는지 확인할 수 있습니다. 클라이언트 측에서는 주로 결제 요청을 사용자에게 보여주고, 사용자의 입력을 받아 서버로 전송하는 역할을 합니다.

📝 질문 2: 원스토어의 PNS(Payment Notification Service)란 무엇이고 어떻게 구현하나요? 메시지 수신 서버의 관점에서 Java SpringFramework로 구현한 코드를 예제를 알려주세요
------------------------------------------------------------
원스토어의 PNS(Payment Notification Service)는 결제 관련 알림을 개발자 서버로 전송하는 서비스입니다. 주로 구독 기반 결제 알림을 포함하며, 이 알림은 결제 이벤트의 실시간 업데이트를 제공합니다. 개발자 서버 측에서는 이 알림 메시지를 수신하고 처리하는 로직을 구현해야 합니다.

### PNS 메시지 수신 서버 구현 예시 (Java Spring Framework)

아래는 Java Spring Framework를 사용하여 PNS 메시지를 수신하고 처리하는 간단한 예제입니다. 이 예제에서는 `SubscriptionNotificationController`를 통해 HTTP POST 요청을 처리하며, 메시지 내용을 파싱하고 로그를 남기는 방식으로 구현하였습니다.

####